# Real-Time_People_Counting
1. use keycard swipes timestamps to identify user-device sessions.  
2. count user-device pairs during each session to generate coocurrence matrix.
3. #people = SUM(p(user|device))


In [122]:
import os 
import re
import glob
import spark
import pandas as pd
import functools
import requests
import json
import time
import datetime
import numpy as np
from collections import defaultdict
from datetime import timedelta
from copy import deepcopy
pj = os.path.join
datapath = '/Users/kanghuang/Documents/work/WeFi/data/aplog'

In [125]:
def get_ap_log(bids, days=60):
    df_list = []
    c = 0
    cur_date = pd.to_datetime('now')
    start_date = cur_date - timedelta(days=days)
    start_date = start_date.replace(minute=0, hour=0, second=0)
    end_date = cur_date.replace(minute=0, hour=0, second=0)
    for path in glob.glob(pj(datapath, '*/*.parquet')):
        ap_log = pd.read_parquet(path)
        try:
            ap_log['localTimestamp'] = pd.to_datetime(ap_log['localTimestamp'])
            ap_log = ap_log[(ap_log['buildingUUID'].isin(bids)) 
                           &(ap_log['localTimestamp'] > start_date) 
                           &(ap_log['localTimestamp'] < end_date)
                           &(ap_log['ssid'] != "")]
            if len(ap_log) == 0:
                c += 1
            df_list.append(ap_log)
        except:
            print (path)
    df = pd.concat(df_list)
    print (c)
    return df


bids = [
'f82815b0-3531-0131-c585-6e74ab010f6f',#59
'3f53df10-8251-0130-f786-123138068411',#54
'69fec472-7e19-11e7-9d13-0642b0acf810',#67 
'deafa8c9-0e12-11e6-9312-063c4950d72f',#48
'7b21b409-ffde-46af-8281-7bb1a94f5fb9',#51
'3f53c7b0-8251-0130-f786-123138068411',#53
'3f548ef0-8251-0130-f786-123138068411',#56
'c778017e-c4da-11e7-9d13-0642b0acf810',#74
]

all_ap_df = get_ap_log(set(bids))

# 'abnormal'
# ae301ff2d6f111ea8bde16245ec60c24, 5ad0f61cd55f11ea8bde16245ec60c24, acf38bb0d6f111ea95420242ac130004,
# 83118d7ad62811eaaaea0242ac130004, 2ef054dad49611ea8bde16245ec60c24, 584d7852d55f11ea8bde16245ec60c24,
# 2f10947ad49611ea87f90242ac130004, af5b284ad6f111ea835f0242ac130004, 83ca466cd62811ea8bde16245ec60c24,
# 81ed97d6d62811ea87ca0242ac130004, f716d86ed69711ea8bde16245ec60c24, 83cc7572d62811ea8bde16245ec60c24,
# abce3212d6f111ea8bde16245ec60c24, 02acdcb4d3cd11ea9d7f0242ac130004, 2fe33f38d49611ea8bde16245ec60c24,
# 2ca1379ed49611eab69c0242ac130004, fcd7b26ed69c11ea8bde16245ec60c24, 

/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00018-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00119-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00090-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00191-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00150-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00051-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00124-f38bbf0e-32d1-

/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00103-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00076-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00177-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00141-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00040-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00135-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00034-f38bbf0e-32d1-

/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00074-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00175-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00143-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00042-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00137-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00036-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00083-f38bbf0e-32d1-

/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00131-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00011-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00110-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00099-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00198-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00065-f38bbf0e-32d1-44f7-8b31-5c4623034339-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/ae301ff2d6f111ea8bde16245ec60c24/part-00164-f38bbf0e-32d1-

/Users/kanghuang/Documents/work/WeFi/data/aplog/5ad0f61cd55f11ea8bde16245ec60c24/part-00132-d7dcda00-d3d5-4752-94c2-495fc7881944-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/5ad0f61cd55f11ea8bde16245ec60c24/part-00033-d7dcda00-d3d5-4752-94c2-495fc7881944-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/5ad0f61cd55f11ea8bde16245ec60c24/part-00086-d7dcda00-d3d5-4752-94c2-495fc7881944-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/5ad0f61cd55f11ea8bde16245ec60c24/part-00187-d7dcda00-d3d5-4752-94c2-495fc7881944-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/5ad0f61cd55f11ea8bde16245ec60c24/part-00177-d7dcda00-d3d5-4752-94c2-495fc7881944-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/5ad0f61cd55f11ea8bde16245ec60c24/part-00076-d7dcda00-d3d5-4752-94c2-495fc7881944-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/5ad0f61cd55f11ea8bde16245ec60c24/part-00103-d7dcda00-d3d5-

...

/Users/kanghuang/Documents/work/WeFi/data/aplog/2ca1379ed49611eab69c0242ac130004/part-00147-25a55f7f-b54e-41b9-a0ba-be3a13fe778a-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/2ca1379ed49611eab69c0242ac130004/part-00046-25a55f7f-b54e-41b9-a0ba-be3a13fe778a-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/2ca1379ed49611eab69c0242ac130004/part-00133-25a55f7f-b54e-41b9-a0ba-be3a13fe778a-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/2ca1379ed49611eab69c0242ac130004/part-00032-25a55f7f-b54e-41b9-a0ba-be3a13fe778a-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/2ca1379ed49611eab69c0242ac130004/part-00004-25a55f7f-b54e-41b9-a0ba-be3a13fe778a-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/2ca1379ed49611eab69c0242ac130004/part-00105-25a55f7f-b54e-41b9-a0ba-be3a13fe778a-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/2ca1379ed49611eab69c0242ac130004/part-00070-25a55f7f-b54e-

/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00025-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00018-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00119-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00090-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00191-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00039-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00138-c8367418-f01d-

/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00177-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00033-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00132-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00047-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00146-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00086-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00066-c8367418-f01d-

/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00059-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00064-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00165-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00098-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00199-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00010-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00111-c8367418-f01d-

/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00095-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00020-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00121-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00054-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00155-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00075-c8367418-f01d-433f-b546-980eaa9affa4-c000.snappy.parquet
/Users/kanghuang/Documents/work/WeFi/data/aplog/fcd7b26ed69c11ea8bde16245ec60c24/part-00174-c8367418-f01d-

In [126]:
bid = 'f82815b0-3531-0131-c585-6e74ab010f6f'
valid_ap_df = all_ap_df[(all_ap_df['buildingUUID'] == bid) & (all_ap_df['floorNumber'] != 999)]
valid_ap_df.loc[:,'date'] = pd.to_datetime(valid_ap_df['date'])
valid_ap_df.loc[:,'type'] = 'device'
valid_ap_df = valid_ap_df.rename(columns={'deviceMac': 'uuid', 'floorNumber': 'floor_number'})

In [130]:
valid_ap_df.date.unique()

array(['2020-08-16T00:00:00.000000000', '2020-08-09T00:00:00.000000000',
       '2020-08-13T00:00:00.000000000', '2020-09-04T00:00:00.000000000',
       '2020-08-10T00:00:00.000000000', '2020-08-06T00:00:00.000000000',
       '2020-08-19T00:00:00.000000000', '2020-08-15T00:00:00.000000000',
       '2020-08-18T00:00:00.000000000', '2020-08-30T00:00:00.000000000',
       '2020-08-31T00:00:00.000000000', '2020-08-23T00:00:00.000000000',
       '2020-08-24T00:00:00.000000000', '2020-09-06T00:00:00.000000000',
       '2020-08-21T00:00:00.000000000', '2020-08-14T00:00:00.000000000',
       '2020-08-05T00:00:00.000000000', '2020-08-22T00:00:00.000000000',
       '2020-07-28T00:00:00.000000000', '2020-07-29T00:00:00.000000000',
       '2020-08-02T00:00:00.000000000', '2020-08-04T00:00:00.000000000',
       '2020-07-31T00:00:00.000000000', '2020-08-03T00:00:00.000000000',
       '2020-07-27T00:00:00.000000000', '2020-08-01T00:00:00.000000000',
       '2020-07-30T00:00:00.000000000', '2020-08-28

In [220]:
def filter_outlier(row):
    # overnight
    timestamp = pd.to_datetime(row.name)
    earlyTimestamp = timestamp.replace(minute=0, hour=6, second=0)
    lateTimestamp = timestamp.replace(minute=0, hour=23, second=0)
    overnight = (row['localTimestamp'] < earlyTimestamp) | (row['localTimestamp'] > lateTimestamp)
    row =  row[~overnight]
#     row['date'] = pd.to_datetime(row['date'])
#     row['dow'] = row.date.apply(lambda x: x.weekday())
#     row = row[row.dow >= 5]
    return row

keycard_swipes_df = pd.read_csv('keycard_swipes.csv')
keycard_swipes_df['date'] = keycard_swipes_df.apply(lambda row: row['KEYCARD_SWIPE_LOCAL'].split()[0], axis=1)
keycard_swipes_df = keycard_swipes_df.rename(columns={'KEYCARD_SWIPE_LOCAL': 'localTimestamp', 'CUSTOMER_UUID': 'uuid'})
keycard_swipes_df['localTimestamp'] = pd.to_datetime(keycard_swipes_df['localTimestamp'])
keycard_swipes_df['date'] = pd.to_datetime(keycard_swipes_df['date'])
keycard_swipes_df = keycard_swipes_df[keycard_swipes_df['date'].isin(valid_ap_df.date.unique())]
valid_keycard_df = deepcopy(keycard_swipes_df)
valid_keycard_df = valid_keycard_df[(valid_keycard_df.READER_NAME != 'Proxy Reader') & (valid_keycard_df.ACTIVITY == 'Access granted')]
valid_keycard_df = valid_keycard_df.groupby('date').apply(filter_outlier).reset_index(drop=True)
# valid_keycard_df['floor_number'] = valid_keycard_df.READER_NAME.str.extract(r'(\d{1,2})th ').fillna('') + valid_keycard_df.READER_NAME.str.extract(r'Office (\d{1,2})\d{2}').fillna('')
# valid_keycard_df = valid_keycard_df[(valid_keycard_df.floor_number != '')]
# cross_floor_keycard = valid_keycard_df.groupby('uuid').apply(lambda row: len(row['HOTSTAMP'].unique()))
# cross_floor_keycard = cross_floor_keycard[cross_floor_keycard > 1].index
# valid_keycard_df = valid_keycard_df[(valid_keycard_df.floor_number != '') & (~valid_keycard_df.uuid.isin(cross_floor_keycard))]
valid_keycard_df['type'] = 'keycard'
valid_keycard_df = valid_keycard_df[valid_keycard_df.date.isin(valid_ap_df.date.unique()) & (valid_keycard_df.LOCATION_UUID == bid)]


### Get stationary groups

In [377]:
def filter_stationary(row, keycard_flag):
    # overnight
    timestamp = pd.to_datetime(row.name)
    earlyTimestamp = timestamp.replace(minute=0, hour=6, second=0)
    lateTimestamp = timestamp.replace(minute=0, hour=23, second=0)
    if keycard_flag:
        overnight = (row['localTimestamp'] < row['min']) | (row['localTimestamp'] > lateTimestamp)
    else:
        overnight = (row['localTimestamp'] < earlyTimestamp) | (row['localTimestamp'] > lateTimestamp)
    row =  row[overnight]
#     row['date'] = pd.to_datetime(row['date'])
#     row['dow'] = row.date.apply(lambda x: x.weekday())
#     row = row[row.dow >= 5]
    return row

def smallest_stationary_group(ap_log):
    day_n = len(ap_log.date.unique())
    ap_log = ap_log.groupby('uuid').apply(lambda row: len(row['date'].unique()))
    ap_log = ap_log[ap_log == day_n]
    return ap_log.index.tolist()

def largest_stationary_group(ap_log):
    day_n = len(ap_log.date.unique())
    ap_log = ap_log.groupby('uuid').apply(lambda row: len(row['date'].unique()))
    ap_log = ap_log[ap_log > 1]
    return ap_log.index.tolist()

def gen_stationary_group(valid_ap_df, keycard_swipes_df=None):
    #overlap, vocab = cooccurrence_model
    keycard_flag=False
    if keycard_swipes_df is not None:
        boundary_df = keycard_swipes_df.groupby('date')['localTimestamp'].agg(['min'])
        valid_ap_df = pd.merge(valid_ap_df, boundary_df, on='date')
        keycard_flag = True
        
    overnight_ap_log = valid_ap_df.groupby('date').apply(lambda df: filter_stationary(df, keycard_flag)).reset_index(drop=True)
    #stationary_group = smallest_stationary_group(overnight_ap_log)
    stationary_group = largest_stationary_group(overnight_ap_log)
#     still_ids = [vocab[uuid] for uuid in stationary_group]
#     row, col = np.where(overlap > stationary_thresh)
#     new_still_ids = []
#     for r, c in zip(row, col):
#         if not r in still_ids:
#             continue
#         if overlap[c][r] > stationary_thresh and not c in still_ids:
#             new_still_ids.append(c)
#     rev_vocab = {i: uuid for uuid, i in vocab.items()}
#     print ('thresh %f: add extra %d to stationary_group' % (stationary_thresh, len(new_still_ids)))
#     for i in new_still_ids:
#         stationary_group.append(rev_vocab[i])
    return stationary_group

In [14]:
'''
This function calculate the cooccurrence count within a single timespan.
There are two groups to maintain the state, left group records device ids from previous time window,
right group records new incoming device ids.
There are three conditions to update the cooccurrence pairs:
1. left device is from left group, right device is from and only exists in right group without occur in left device's timespan 
2. left device is from right group, right device is from and only exists in left group.
3. both devices are from right group
'''
def count_cooccurrence(l, r, rr, cooccurrence, obsolete_index, devices):
    """
    l: left index of left group
    """
    left_freq = defaultdict(int)
    right_freq = defaultdict(int)
    for i in devices[l: r]:
        left_freq[i] += 1
    for i in devices[r: rr]:
        right_freq[i] += 1
        
    rd = right_freq.keys()
    ld = left_freq.keys()
    prev = -1
    obsolete_devices = set()
    # condition 1
    for i in rd: 
        if (not i in ld):
            for k in range(l, r):
                j = devices[k]
                st = obsolete_index[k]
                if prev != st:
                    obsolete_devices = set(devices[st: k]) 
                    prev = st
                if not i in obsolete_devices: 
                    cooccurrence[j, i] += 1
    # condition 2               
    for i in rd: 
        for j in ld:
            if not j in rd: 
                cooccurrence[i, j] += right_freq[i]
    
    # condition 3
    for i in rd:
        for j in rd: 
            if i != j:
                cooccurrence[i, j] += right_freq[i]    
                
"""
This function slides the time window from left to right in steps of timestamps,
every time left index will try to shift one timestamp to right direction, and right index will find the 
right most index within the time window starts from left index.
It will also maintain every device left most index within time window
"""                
def create_context(timespan, df, cooccurrence, device_vocab):
    df = df.sort_values('localTimestamp')
    timestamps = df.localTimestamp.tolist()
    types = df.type.tolist()
    devices = [device_vocab[mac] for mac in df['uuid'].tolist()]
    l, r = 0, 0 
    obsolete_index = []
    while True:
        ll = l
        while ll < r:
            assert timestamps[ll] >= lt
            if timestamps[ll] != lt:
                break
            ll += 1
        if ll == len(timestamps):
            break
        l = ll
        lt = timestamps[l]
        rr = r
        while (rr < len(timestamps)) and (timestamps[rr] < lt + timedelta(minutes=timespan)):
            rr += 1
        if r == rr: continue
        obsolete_index.extend([l]*(rr-r))
        count_cooccurrence(l, r, rr, cooccurrence, obsolete_index, devices)
        r = rr
    assert len(obsolete_index) == len(devices)
        
def cooccurrence_v2(daily_ap_df, timespan, group_columns=['apMac', 'floor_number']):
    # create device to id mapping
    device_vocab = {mac: i for i, mac in enumerate(daily_ap_df['uuid'].unique().tolist())}
    n = len(device_vocab)
    cooccurrence = np.zeros((n, n))
    device_freq = np.zeros((n, 1))
    for mac in daily_ap_df['uuid'].tolist():
        device_freq[device_vocab[mac]] += 1

    for _, df in daily_ap_df.groupby(group_columns):
        create_context(timespan, df, cooccurrence, device_vocab)
    return cooccurrence / device_freq, device_vocab #, device_freq


In [15]:
# Union find
class Union:
    def __init__(self, n):
        self.par = [-1]*n
        
    def find(self, x):
        par = self.par
        if par[x] < 0:
            return x
        else:
            par[x] = self.find(par[x])
            return par[x]

    def unite(self, x, y):
        par = self.par
        x = self.find(x)
        y = self.find(y)

        if x == y:
            return False
        else:
            if par[x] > par[y]:
                x,y = y,x
            par[x] += par[y]
            par[y] = x
            return True

    def same(self, x,y):
        return self.find(x) == self.find(y)

    def size(self, x):
        return -self.par[self.find(x)]

def gen_device_groups(overlap, overlap_thresh):
    row, col = np.where(overlap > overlap_thresh)
    n, m = overlap.shape
    assert n == m
    uf = Union(n)
    undirect_graph = set()
    for x, y in zip(row, col):
        undirect_graph.add((x, y))
    k = 0
    for (x, y) in undirect_graph:
        if (y, x) in undirect_graph:
            uf.unite(x, y) 
            k += 1
    print ('group size %d'%k)
    return uf

In [369]:
timespan = 5
def find_right_closest(arr, target): 
    n = len(arr)
    # Corner cases 
    if (target <= arr[0]): 
        return 0 
    if (target >= arr[n - 1]): 
        return n - 1 
    # Doing binary search 
    i = 0; j = n; mid = 0
    while (i < j):  
        mid = (i + j) // 2
        if (arr[mid] == target): 
            i = mid + 1
        # If target is less than array  
        # element, then search in left 
        if (target < arr[mid]) : 
            # If target is greater than previous 
            # to mid, return closest of two 
            if (mid > 0 and target > arr[mid - 1]): 
                return mid
            # Repeat for left half  
            j = mid 
        # If target is greater than mid 
        else : 
            if (mid < n - 1 and target < arr[mid + 1]): 
                return mid+1
            # update i 
            i = mid + 1
    # Only single element left after search 
    return mid 

def second_swipe_time(keycard_day_df, ap_timestamp, uuid):
    keycard_timestamps = keycard_day_df.localTimestamp.tolist()
    keycard_uuids = keycard_day_df.uuid.tolist()
    i = find_right_closest(keycard_timestamps, ap_timestamp)
    while i < len(keycard_uuids) and keycard_uuids[i] == uuid:
        i += 1
    if i == len(keycard_uuids):
        ap2nd_timestamp = keycard_timestamps[i-1].replace(minute=0, hour=23, second=0)
    else:
        ap2nd_timestamp = keycard_timestamps[i]
    return ap2nd_timestamp

def session(row, metadata, first=True):
    uuid, date = metadata['index']
    keycard_df, good_ap_df, graph = metadata['keycard'], metadata['ap'], metadata['graph']
    if not uuid in graph:
        graph[uuid] = defaultdict(int)
    ap_timestamp = row['localTimestamp']
    ap_day_df = good_ap_df[good_ap_df['date'] == date]
    keycard_day_df = keycard_df[keycard_df['date']==date]
    rel_ap_df = ap_day_df[ap_day_df['localTimestamp'] < ap_timestamp]
    cond = (ap_day_df['localTimestamp'] > ap_timestamp)
    if first:
        ap2nd_timestamp = second_swipe_time(keycard_day_df, ap_timestamp, uuid)
    else:
        ap2nd_timestamp = ap_timestamp + timedelta(minutes=timespan)
    cond &= (ap_day_df['localTimestamp'] < ap2nd_timestamp)
    rel2nd_ap_df = ap_day_df[cond]
    outof_bag = rel_ap_df.uuid.unique()
    in_bag = rel2nd_ap_df.uuid.unique() 
    k = 0
    for i in in_bag:
        if not i in outof_bag:
            k += 1
            if first:
                graph[uuid][i] += 1
            else:
                decay = len(keycard_df[(keycard_df['date'] == date) & (keycard_df['localTimestamp'] < ap2nd_timestamp)].uuid.unique())
                graph[uuid][i] += 1.0/decay
#     print (k)
                    
def accumulate(rows, metadata):
    metadata['index'] = rows.name
    rows = rows.sort_values('localTimestamp').reset_index(drop=True)
    for i, (_, row) in enumerate(rows.iterrows()):
        session(row, metadata, first=(i==0))
    
        
def gen_keycard_device_graph(good_ap_df, valid_keycard_df):
    kd_graph = {}
    #good_ap_df = good_ap_df.sort_values('localTimestamp').reset_index(drop=True)
    valid_keycard_df = valid_keycard_df.sort_values('localTimestamp').reset_index(drop=True)
    metadata = {
        'keycard': valid_keycard_df,
        'ap': good_ap_df,
        'graph': kd_graph
    }
    
    valid_keycard_df.groupby(['uuid', 'date']).apply(lambda rows: accumulate(rows, metadata))
    return kd_graph

def gen_prob(kd_graph):
    kv, dv = {}, {}
    for kid in kd_graph:
        if not kid in kv:
            kv[kid] = len(kv)
        for did in kd_graph[kid]:
            if not did in dv:
                dv[did] = len(dv)
    cocur = np.zeros((len(kv), len(dv)))
    kcur = np.zeros((len(kv), 1))
    dcur = np.zeros((1, len(dv)))
    for kid in kd_graph:
        for did in kd_graph[kid]:
            #kcur[kv[kid], 0] += kd_graph[kid][did]
            cocur[kv[kid], dv[did]] = kd_graph[kid][did]
            #dcur[0, dv[did]] += kd_graph[kid][did]
    probs = cocur / (np.sum(cocur, axis=0, keepdims=True)+1e-3)
    return probs, kv, dv

def soft_count(df, model, count_thresh):
    probs, kv, dv = model
    freq = df[['uuid', 'date']].groupby('uuid').count()['date']
#     print (sorted(freq))
#     outlier = freq[freq < 30*2].index
    dids = df.uuid.unique()  
#     print (df.name, len(outlier))
    candis = np.array([int(dv[i]) for i in dids if i in dv])
    count = np.sum(probs[:, candis], axis=1)
    count[count > count_thresh] = 1
    count = np.sum(count)
    return count

def filter_low_rssi(valid_ap_df):
    rssi = sorted(valid_ap_df.rssi.tolist(), reverse=True)
    #print ('rssi: ', rssi[round(len(rssi)*2.0/3)])
    rssi_cond = (valid_ap_df['rssi'] < 0) & (valid_ap_df['rssi'] > rssi[round(len(rssi)*2.0/3)])
    return valid_ap_df[rssi_cond]

def filter_weekend(df):
    df['date']  = pd.to_datetime(df['date'])
    df['dow'] = df.date.apply(lambda x: x.weekday())
    return df[df.dow < 5]

def preprocess(valid_keycard_df, valid_ap_df):
    good_ap_df = filter_low_rssi(valid_ap_df)
    stationary_group = gen_stationary_group(good_ap_df, valid_keycard_df)
    good_ap_df = good_ap_df[~good_ap_df.uuid.isin(stationary_group)]
    return good_ap_df

def cooccurrence_v1(valid_keycard_df, valid_ap_df):
    kd_graph = gen_keycard_device_graph(good_ap_df, valid_keycard_df)
    probs, kv, dv = gen_prob(kd_graph)
    return probs, kv, dv

def simulation(valid_keycard_df, valid_ap_df, count_thresh):
    #overlap, vocab = cooccurrence_model
    good_ap_df = preprocess(valid_keycard_df, valid_ap_df)
    kd_graph = gen_keycard_device_graph(good_ap_df, valid_keycard_df)
    probs, kv, dv = gen_prob(kd_graph)
    #uf = gen_device_groups(overlap, overlap_thresh)
    metadata = {
        'prob': probs,
        #'keycard': valid_keycard_df,
        'kv': kv,
        'dv': dv,
        #'uf': None,
        'thresh': count_thresh,
    }
    
    estimate = good_ap_df.groupby('date').apply(lambda df: soft_count(df, metadata))
    truth = valid_keycard_df.groupby('date').apply(lambda rows: len(rows.uuid.unique()))
#     print (good_ap_df.groupby('date').apply(lambda rows: len(rows.uuid.unique())))
#     print (estimate)
#     print (truth)
    #return -abs(np.sum((estimate-truth) / truth) / len(truth) * 100)
    return good_ap_df, estimate, truth

In [378]:
st = time.time()
good_ap_df = preprocess(valid_keycard_df, valid_ap_df)
model_v1 = cooccurrence_v1(valid_keycard_df, good_ap_df)
print ('running time %d'% (time.time()-st))

running time 313


In [360]:
device = good_ap_df.groupby('date').apply(lambda rows: len(rows.uuid.unique()))
device

date
2020-07-27    52
2020-07-28    58
2020-07-29    64
2020-07-30    55
2020-07-31    41
2020-08-01     7
2020-08-02     1
2020-08-03    43
2020-08-04    28
2020-08-05    63
2020-08-06    54
2020-08-07    47
2020-08-08     1
2020-08-09     2
2020-08-10    51
2020-08-11    66
2020-08-12    55
2020-08-13    78
2020-08-14    41
2020-08-15    10
2020-08-16     2
2020-08-17    51
2020-08-18    52
2020-08-19    69
2020-08-21    40
2020-08-22     5
2020-08-23     4
2020-08-24    51
2020-08-25    53
2020-08-27    63
2020-08-28    53
2020-08-29     4
2020-08-30     4
2020-08-31    73
2020-09-01    69
2020-09-04    27
2020-09-05     2
2020-09-06     3
dtype: int64

In [ ]:
from bayes_opt import BayesianOptimization    
from bayes_opt import SequentialDomainReductionTransformer

black_box_function = functools.partial(simulation, valid_keycard_df, valid_ap_df, model)
pbounds = {'count_thresh': (0, 1)}
bounds_transformer = SequentialDomainReductionTransformer()
mutating_optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    verbose=1,
    random_state=7,
    bounds_transformer=bounds_transformer
)
st = time.time()
mutating_optimizer.maximize(
    init_points=2,
    n_iter=30,
)
print ('running time %d'% (time.time()-st))

In [400]:
estimate_sf = good_ap_df.groupby('date').apply(lambda df: soft_count(df, model_v1, .5))
truth = valid_keycard_df.groupby('date').apply(lambda rows: len(rows.uuid.unique()))
device = good_ap_df.groupby('date').apply(lambda rows: len(rows.uuid.unique()))

In [397]:
def filter_series_weekend(series):
    df = series.reset_index(drop=False)
    df['dow'] = df.date.apply(lambda x: x.weekday())
    return df[df.dow < 5]

def error_rate(df1, df2):
    return np.sum(np.abs((df1[0]-df2[0]) / df2[0])) * 100 /  len(df2[0])

In [402]:
estimate_sf_ = filter_series_weekend(estimate_sf)
truth_ = filter_series_weekend(truth)
device_ = filter_series_weekend(device)

In [408]:
error_rate(estimate_sf_, truth_)
np.sum(np.abs(estimate_sf-truth) / truth) / len(truth) * 100

20.604087337573127

In [419]:
truth

date
2020-07-27    41
2020-07-28    50
2020-07-29    49
2020-07-30    45
2020-07-31    36
2020-08-01    10
2020-08-02     7
2020-08-03    36
2020-08-04    25
2020-08-05    46
2020-08-06    42
2020-08-07    32
2020-08-08     1
2020-08-09     5
2020-08-10    30
2020-08-11    50
2020-08-12    42
2020-08-13    48
2020-08-14    29
2020-08-15     7
2020-08-16     2
2020-08-17    39
2020-08-18    42
2020-08-19    28
2020-08-21    29
2020-08-22     5
2020-08-23     6
2020-08-24    34
2020-08-25    43
2020-08-27    38
2020-08-28    38
2020-08-29     6
2020-08-30     6
2020-08-31    47
2020-09-01    53
2020-09-04    36
2020-09-05     7
2020-09-06     3
dtype: int64

In [375]:
(estimate_sf_[0] / device_[0]), (truth_[0] /  device_[0])

(0     0.521386
 1     0.520913
 2     0.502043
 3     0.514686
 4     0.578387
 7     0.560185
 8     0.580291
 9     0.506636
 10    0.582700
 11    0.557334
 14    0.541869
 15    0.545663
 16    0.532367
 17    0.507716
 18    0.542540
 21    0.586114
 22    0.598011
 23    0.503906
 24    0.537540
 27    0.537711
 28    0.573700
 29    0.556015
 30    0.573202
 33    0.560528
 34    0.557335
 35    0.602787
 Name: 0, dtype: float64, 0     0.500000
 1     0.520833
 2     0.485149
 3     0.500000
 4     0.486486
 7     0.467532
 8     0.431034
 9     0.464646
 10    0.506024
 11    0.405063
 14    0.379747
 15    0.505051
 16    0.466667
 17    0.440367
 18    0.414286
 21    0.481481
 22    0.512195
 23    0.274510
 24    0.432836
 27    0.409639
 28    0.511905
 29    0.426966
 30    0.481013
 33    0.451923
 34    0.524752
 35    0.692308
 Name: 0, dtype: float64)

### Strong connected components

In [35]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
import scipy as spy

In [337]:
good_ap_df = filter_low_rssi(valid_ap_df)
stationary_group = gen_stationary_group(good_ap_df)
good_ap_df = good_ap_df[~good_ap_df.uuid.isin(stationary_group)]


In [351]:
stationary_thresh = .5
overlap, vocab = model_v2
still_ids = [vocab[uuid] for uuid in stationary_group]
row, col = np.where(overlap > stationary_thresh)
new_still_ids = []
print (len(row))
for r, c in zip(row, col):
    if not r in still_ids:
        continue
    print (overlap[c][r], overlap[r][c])
    if overlap[c][r] > stationary_thresh and not c in still_ids:
        new_still_ids.append(c)
rev_vocab = {i: uuid for uuid, i in vocab.items()}
print ('thresh %f: add extra %d to stationary_group' % (stationary_thresh, len(new_still_ids)))
# for i in new_still_ids:
#     stationary_group.append(rev_vocab[i])

3207
0.5430435273933902 0.6314274636547077
0.8575419454403448 0.7975172658416042
0.715050888350322 0.9429946577538039
0.44810966771588917 0.5795358979169591
0.4170757940022776 0.5163348871626063
0.3382265587662502 0.5463527885563028
0.9810372473166776 0.8923480780540985
0.4853895258754261 0.6917069831837345
0.6089316396740796 0.7160524555481231
0.15702276953576996 0.976783908653921
0.08705182865545952 0.9252026196661637
0.13438659600697062 0.9511410481775897
0.25363224735012585 0.9893937744038082
0.06802351205666884 0.5216184683353298
0.8923480780540985 0.9810372473166776
0.03728544811926619 0.9307472903593839
0.01911735892962104 0.5940673131774101
0.41730912891862604 0.633035127983408
0.7960219031041486 0.6961978817884519
0.2428906519719234 0.6953039158956889
0.7987170739870935 0.6898230922767848
0.06597847393555924 0.5950241879751209
0.0592492848190706 0.8078783690393918
0.18407392425096922 0.6190481491046065
0.3139664991220389 0.731457652475133
0.15669927765893557 0.8580905990364837

In [430]:
def ssg(df, model):
    overlap, vocab = model
    candidate = []
    for i in df.uuid.unique():
        if i in vocab:
            candidate.append(vocab[i])
    candidate = np.array(candidate)
    graph = overlap[candidate][:,candidate]
    graph[graph > 0.5] = 1
    graph[graph < 0.5] = 0
    graph = csr_matrix(graph)
    n_components, labels = connected_components(csgraph=graph, connection='strong', return_labels=True)
    return n_components

st = time.time()
# good_ap_df = preprocess(valid_keycard_df, valid_ap_df)
#model_v2 = cooccurrence_model(valid_ap_df, 5)   
estimate_ssg = good_ap_df.groupby('date').apply(lambda df: ssg(df, model_v2))
print ('running time %d'% (time.time()-st))

running time 1


In [431]:
estimate_ssg_ = filter_series_weekend(estimate_ssg)

In [432]:
error_rate(estimate_ssg_, truth_)

14.149939368112776

In [250]:
Y = np.expand_dims(np.array(truth_[0].tolist()), -1)
X = np.expand_dims(np.array(device_[0].tolist()), -1)
a, _, _, _ = np.linalg.lstsq(X, Y)

/Users/kanghuang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [285]:
error_rate((a*X).T, Y.T)

10.996851383035295

In [274]:
np.hstack([(a*X), Y])

array([[37.97075751, 41.        ],
       [44.45356976, 50.        ],
       [46.76885985, 49.        ],
       [41.67522165, 45.        ],
       [34.26629336, 36.        ],
       [35.65546741, 36.        ],
       [26.85736506, 25.        ],
       [45.84274382, 46.        ],
       [38.43381552, 42.        ],
       [36.58158345, 32.        ],
       [36.58158345, 30.        ],
       [45.84274382, 50.        ],
       [41.67522165, 42.        ],
       [50.473324  , 48.        ],
       [32.41406128, 29.        ],
       [37.50769949, 39.        ],
       [37.97075751, 42.        ],
       [47.23191787, 28.        ],
       [31.02488723, 29.        ],
       [38.43381552, 34.        ],
       [38.89687354, 43.        ],
       [41.21216363, 38.        ],
       [36.58158345, 38.        ],
       [48.15803391, 47.        ],
       [46.76885985, 53.        ],
       [24.07901695, 36.        ]])

In [391]:
percentage = (estimate_sf-truth) / truth
percentage_ =  filter_series_weekend(percentage)

In [433]:
tmp_df = pd.concat([estimate_sf, truth, percentage, estimate_ssg, device], axis=1).reset_index(drop=False)
tmp_df['dow'] = tmp_df.apply(lambda row: row.date.weekday(), axis=1)
# tmp_df[tmp_df['dow']<5]
tmp_df

,date,0,1,2,3,4,dow
0,2020-07-27,37.515294,41,-0.118172,42,52,0
1,2020-07-28,49.037416,50,-0.113413,47,58,1
2,2020-07-29,48.605809,49,-0.113822,42,64,2
3,2020-07-30,42.101940,45,-0.145884,39,55,3
4,2020-07-31,38.659566,36,-0.005729,32,41,4
5,2020-08-01,6.315209,10,-0.368479,7,7,5
6,2020-08-02,1.146082,7,-0.836274,1,1,6
7,2020-08-03,39.124404,36,-0.054456,32,43,0
8,2020-08-04,25.618273,25,-0.146977,24,28,1
9,2020-08-05,46.633154,46,-0.048699,48,63,2


In [418]:
valid_keycard_df[(valid_keycard_df.date == '2020-08-19') & (valid_keycard_df.uuid == 'dcc83210-6b4c-0135-98c0-0242ac11560d')].sort_values('localTimestamp')

,Unnamed: 0,UUID,uuid,ACCOUNT_UUID,LOCATION_UUID,localTimestamp,ACTIVITY,READER_NAME,date,type
16981,1455,459326710FIVETRAN.s2_keycard_workbench_public....,dcc83210-6b4c-0135-98c0-0242ac11560d,NaN,f82815b0-3531-0131-c585-6e74ab010f6f,2020-08-19 07:56:46.035762,Access granted,NY09 9th Fl West Entrance Reader,2020-08-19,keycard
16977,1444,459334937FIVETRAN.s2_keycard_workbench_public....,dcc83210-6b4c-0135-98c0-0242ac11560d,NaN,f82815b0-3531-0131-c585-6e74ab010f6f,2020-08-19 09:12:39.067759,Access granted,NY09 9th Fl West Entrance Reader,2020-08-19,keycard
16975,1440,459336075FIVETRAN.s2_keycard_workbench_public....,dcc83210-6b4c-0135-98c0-0242ac11560d,NaN,f82815b0-3531-0131-c585-6e74ab010f6f,2020-08-19 09:23:45.050345,Access granted,NY09 5th Floor Lobby East Reader,2020-08-19,keycard
16973,1433,459337398FIVETRAN.s2_keycard_workbench_public....,dcc83210-6b4c-0135-98c0-0242ac11560d,NaN,f82815b0-3531-0131-c585-6e74ab010f6f,2020-08-19 09:35:07.605654,Access granted,NY09 5th Floor Lobby West Reader,2020-08-19,keycard
16971,1428,459337498FIVETRAN.s2_keycard_workbench_public....,dcc83210-6b4c-0135-98c0-0242ac11560d,NaN,f82815b0-3531-0131-c585-6e74ab010f6f,2020-08-19 09:35:56.349921,Access granted,NY09 9th Fl West Entrance Reader,2020-08-19,keycard
16959,1397,459337769FIVETRAN.s2_keycard_workbench_public....,dcc83210-6b4c-0135-98c0-0242ac11560d,NaN,f82815b0-3531-0131-c585-6e74ab010f6f,2020-08-19 09:38:36.885743,Access granted,NY09 5th Floor Lobby East Reader,2020-08-19,keycard
16957,1394,459339171FIVETRAN.s2_keycard_workbench_public....,dcc83210-6b4c-0135-98c0-0242ac11560d,NaN,f82815b0-3531-0131-c585-6e74ab010f6f,2020-08-19 09:50:50.506670,Access granted,NY09 5th FL Mothers Room Reader,2020-08-19,keycard
16970,1426,459341538FIVETRAN.s2_keycard_workbench_public....,dcc83210-6b4c-0135-98c0-0242ac11560d,NaN,f82815b0-3531-0131-c585-6e74ab010f6f,2020-08-19 10:11:06.878267,Access granted,NY09 9th Fl West Entrance Reader,2020-08-19,keycard


In [319]:
count_df = valid_ap_df[valid_ap_df.uuid.isin(model[2])].groupby('uuid').count()['date']

In [320]:
sorted(count_df.tolist())

[4,
 6,
 6,
 9,
 14,
 16,
 21,
 24,
 24,
 25,
 28,
 29,
 30,
 36,
 40,
 48,
 48,
 50,
 52,
 54,
 55,
 57,
 61,
 68,
 68,
 77,
 84,
 96,
 96,
 136,
 150,
 170,
 174,
 178,
 182,
 185,
 198,
 202,
 204,
 225,
 234,
 234,
 236,
 260,
 264,
 264,
 264,
 300,
 308,
 320,
 320,
 326,
 334,
 350,
 354,
 357,
 360,
 378,
 378,
 393,
 400,
 400,
 405,
 408,
 410,
 424,
 434,
 438,
 480,
 500,
 502,
 516,
 551,
 569,
 598,
 612,
 628,
 647,
 651,
 678,
 680,
 699,
 710,
 726,
 748,
 748,
 791,
 808,
 826,
 831,
 850,
 892,
 905,
 910,
 916,
 936,
 940,
 945,
 948,
 952,
 953,
 1018,
 1030,
 1068,
 1080,
 1094,
 1100,
 1125,
 1138,
 1160,
 1186,
 1225,
 1227,
 1244,
 1262,
 1269,
 1275,
 1494,
 1510,
 1532,
 1556,
 1587,
 1655,
 1833,
 1850,
 1863,
 1863,
 1899,
 1957,
 2016,
 2073,
 2084,
 2118,
 2165,
 2205,
 2210,
 2226,
 2239,
 2284,
 2322,
 2325,
 2508,
 2510,
 2622,
 2652,
 2698,
 2700,
 2711,
 2736,
 2747,
 2860,
 2882,
 2886,
 2905,
 2906,
 2930,
 2934,
 2990,
 3012,
 3040,
 3056,
 3068,


In [318]:
valid_ap_df.uuid

379611    36204b8aa0762d6fa68e2186a11da90e499e898bd908f2...
379612    36204b8aa0762d6fa68e2186a11da90e499e898bd908f2...
379613    36204b8aa0762d6fa68e2186a11da90e499e898bd908f2...
379614    36204b8aa0762d6fa68e2186a11da90e499e898bd908f2...
379615    36204b8aa0762d6fa68e2186a11da90e499e898bd908f2...
                                ...                        
550088    e8d2dc2d718b7f4be09a1a1126dcb8d38d67564df558ab...
550089    e8d2dc2d718b7f4be09a1a1126dcb8d38d67564df558ab...
550090    e8d2dc2d718b7f4be09a1a1126dcb8d38d67564df558ab...
550091    e8d2dc2d718b7f4be09a1a1126dcb8d38d67564df558ab...
550092    e8d2dc2d718b7f4be09a1a1126dcb8d38d67564df558ab...
Name: uuid, Length: 37453312, dtype: object